# Lab 1: Stochastic version of The Farmer Problem
## Stochastic Programming (MESIO), 28/02/2025

### Arnau Pérez Reverte

#### 1. Set up AMPLpy

In [1]:
from amplpy import AMPL, Environment

ampl = AMPL(Environment(""))
ampl.read("ampl/farmer_stochastic.mod")
ampl.setOption("solver", "cplex")
ampl.option["solver"] = "cplex"

#### 2. Data preparation using Pandas

In [2]:
import pandas as pd

crop_yield = {
    "BAD": {"wheat": {"yield": 2}, "corn": {"yield": 2.4}, "sugar_beet": {"yield": 16}},
    "NORMAL": {
        "wheat": {"yield": 2.5},
        "corn": {"yield": 3},
        "sugar_beet": {"yield": 20},
    },
    "GOOD": {
        "wheat": {"yield": 3},
        "corn": {"yield": 3.6},
        "sugar_beet": {"yield": 24},
    },
}

cplant = {
    "wheat": 150,
    "corn": 230,
    "sugar_beet": 260,
}

yield_data = {
    (crop, scenario): crop_yield[scenario][crop]["yield"]
    for scenario in crop_yield
    for crop in crop_yield[scenario]
}
yield_df = pd.DataFrame.from_dict(yield_data, orient="index", columns=["yield"])
yield_df.index = pd.MultiIndex.from_tuples(yield_df.index, names=["CROP", "SCENARIOS"])

cplant_df = pd.DataFrame.from_dict(cplant, orient="index", columns=["cplant"])
cplant_df.index.name = "CROP"


crop_data = {
    'wheat': {'min_crop': 200, 'psell': 170, 'ppurchase': 238},
    'corn': {'min_crop': 240, 'psell': 150, 'ppurchase': 210}
}
crop_df = pd.DataFrame.from_dict(crop_data, orient='index')
crop_df.index.name = 'CROP1'

pvbelow  = 36
pvabove = 10
land_total  = 500 
pscenario = float(1)/3

#### 3. Data parsing into AMPL

In [3]:
ampl.set['SCENARIOS'] = ['BAD', 'NORMAL', 'GOOD']
ampl.set['CROP1'] = ['wheat', 'corn']
ampl.set['CROP2'] = ['sugar_beet']

ampl.param['yield'] = yield_df
ampl.param['cplant'] = cplant_df
ampl.param['min_crop'] = crop_df[['min_crop']]
ampl.param['psell'] = crop_df[['psell']]
ampl.param['ppurchase'] = crop_df[['ppurchase']]
ampl.param['pvbelow'] = pvbelow
ampl.param['pvabove'] = pvabove
ampl.param['land_total'] = land_total
ampl.param['pscenario'] = pscenario

#### 4. Solution

In [4]:
ampl.solve(solver="cplex")

obj_value = obj = ampl.get_objective("Cost_total").value()
print(f"Objective value: {-obj_value} €")

CPLEX 22.1.1CPLEX 22.1.1: optimal solution; objective -108390
15 simplex iterations
Objective value: 108390.0 €


Export solution as CSV:

In [ ]:
import os

os.makedirs('exports/', exist_ok=True)

ampl.get_variable("x").getValues().to_pandas().to_csv("exports/x.csv")
ampl.get_variable("y").getValues().to_pandas().to_csv("exports/y.csv")
ampl.get_variable("w").getValues().to_pandas().to_csv("exports/w.csv")
ampl.get_variable("wbelow").getValues().to_pandas().to_csv("exports/wbelow.csv")
ampl.get_variable("wabove").getValues().to_pandas().to_csv("exports/wabove.csv")